In [1]:
from dumbcoin import Blockchain
import time

In [2]:
# Initialize a blockchain with 2000 initial coins
dumbcoin = Blockchain(2000)

In [3]:
# Print the blockchain and confirm that it is valid
print(dumbcoin)
print(dumbcoin.verify_blockchain(dumbcoin.last_block))

Block: 0,
Time: 1516384655.2234187,
Proof: 39593,
Transactions: [{'amount': 2000, 'sender': None, 'recipient': 'genesis', 'timestamp': 1516384655.2234187}]

True


In [4]:
# Set up some initial transactions
dumbcoin.add_transaction("genesis", "adam", 1000)
dumbcoin.add_transaction("genesis", "eve", 1000)
# And add them into a new block
dumbcoin.add_block()

# And some more giving
dumbcoin.add_transaction("adam", "cain", 100)
dumbcoin.add_transaction("adam", "abel", 200)
dumbcoin.add_block()

Staging transaction: {'amount': 1000, 'sender': 'genesis', 'recipient': 'adam', 'timestamp': 1516384657.584574}
Validating new transaction: {'amount': 1000, 'sender': 'genesis', 'recipient': 'adam', 'timestamp': 1516384657.584574}
Transaction validated against ledger
Transaction stagged. 1 transactions awaiting mining
Staging transaction: {'amount': 1000, 'sender': 'genesis', 'recipient': 'eve', 'timestamp': 1516384657.5853496}
Validating new transaction: {'amount': 1000, 'sender': 'genesis', 'recipient': 'eve', 'timestamp': 1516384657.5853496}
Transaction validated against ledger
Transaction stagged. 2 transactions awaiting mining
Mining new block with 2 transactions
New block at index 1 mined in 0.15718700000002173s
Block successfully added to blockchain at index 1
Staging transaction: {'amount': 100, 'sender': 'adam', 'recipient': 'cain', 'timestamp': 1516384657.743146}
Validating new transaction: {'amount': 100, 'sender': 'adam', 'recipient': 'cain', 'timestamp': 1516384657.743146}

In [5]:
# Print and verify the new chain
print(dumbcoin)
print(dumbcoin.verify_blockchain(dumbcoin.last_block))

Block: 2,
Time: 1516384657.744046,
Proof: 3156,
Transactions: [{'amount': 100, 'sender': 'adam', 'recipient': 'cain', 'timestamp': 1516384657.743146}, {'amount': 200, 'sender': 'adam', 'recipient': 'abel', 'timestamp': 1516384657.743606}]

Block: 1,
Time: 1516384657.585712,
Proof: 75656,
Transactions: [{'amount': 1000, 'sender': 'genesis', 'recipient': 'adam', 'timestamp': 1516384657.584574}, {'amount': 1000, 'sender': 'genesis', 'recipient': 'eve', 'timestamp': 1516384657.5853496}]

Block: 0,
Time: 1516384655.2234187,
Proof: 39593,
Transactions: [{'amount': 2000, 'sender': None, 'recipient': 'genesis', 'timestamp': 1516384655.2234187}]

True


In [6]:
transactions = dumbcoin.get_settled_transactions(dumbcoin.last_block)
for transaction in transactions:
    print(transaction)

{'amount': 2000, 'sender': None, 'recipient': 'genesis', 'timestamp': 1516384655.2234187}
{'amount': 1000, 'sender': 'genesis', 'recipient': 'adam', 'timestamp': 1516384657.584574}
{'amount': 1000, 'sender': 'genesis', 'recipient': 'eve', 'timestamp': 1516384657.5853496}
{'amount': 100, 'sender': 'adam', 'recipient': 'cain', 'timestamp': 1516384657.743146}
{'amount': 200, 'sender': 'adam', 'recipient': 'abel', 'timestamp': 1516384657.743606}


In [7]:
len(dumbcoin)

3

In [7]:
ledger = dumbcoin.create_ledger(transactions)
print(ledger)
valid_trans = {"sender": "adam", "recipient": "eve", "amount": 200, "timestamp": time.time()}
print(dumbcoin.validate_transaction(valid_trans, transactions))
invalid_trans = {"sender": "adam", "recipient": "cain", "amount": 20000, "timestamp": time.time()}
print(dumbcoin.validate_transaction(invalid_trans, transactions))
really_invalid_trans = {"sender": "lucifer", "recipient": "cain", "amount": 700, "timestamp": time.time()}
print(dumbcoin.validate_transaction(really_invalid_trans, transactions))

{'eve': 1000, 'cain': 100, 'abel': 200, 'adam': 700, 'genesis': 0}
Validating new transaction: {'amount': 200, 'sender': 'adam', 'recipient': 'eve', 'timestamp': 1516384787.278364}
True
Validating new transaction: {'amount': 20000, 'sender': 'adam', 'recipient': 'cain', 'timestamp': 1516384787.2787137}
False
Validating new transaction: {'amount': 700, 'sender': 'lucifer', 'recipient': 'cain', 'timestamp': 1516384787.2790515}
False
